# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
databases = mongo.list_database_names()
databases

['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'uk_food']

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our new database
db = mongo['uk_food']
print(db.list_collection_names())

['establishments']


In [8]:
# review a document in the establishments collection
collection = db['establishments']
pprint(collection.find_one())

{'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.638929767669,
 'FHRSID': 1349980,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2021-06-14T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67550779c88e03eb321f31d6'),
 'geocode': {'latitude': '51.150879', 'longitude': '1.378536'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1349980',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00

In [10]:
# assign the collection to a variable
establishments = db['establishments']
establishments

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [13]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('67550e8a4b7ac6bf8f6fb060'), acknowledged=True)

In [19]:
# Check that the new restaurant was inserted
if new_restaurant:
    print("The new restaurant was inserted successfully!")
    pprint(new_restaurant)
else:
    print("The new restaurant was not found.")

The new restaurant was inserted successfully!
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67550e584b7ac6bf8f6fb05f'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [26]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0} 
)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [33]:
# Update the new restaurant with the correct BusinessTypeID
    
update = establishments.update_one(
        {"BusinessName": "Penang Flavours"},  # Filter to find the new restaurant
        {"$set": {"BusinessTypeID": 1}}  # Update the BusinessTypeID
    )

In [34]:
# Confirm that the new restaurant was updated
pprint(update)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents that contain the Dover Local Authority: {dover_count}")

Number of documents that contain the Dover Local Authority: 994


In [37]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [38]:
# Check if any remaining documents include Dover
new_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover after deletion: {new_count}")

Number of establishments in Dover after deletion: 0


In [42]:
# Check that other documents remain with 'find_one'

other_document = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
pprint(other_document)


{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67550779c88e03eb321f34b6'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [43]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"},
               "geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)

UpdateResult({'n': 38787, 'nModified': 38787, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [48]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [49]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$exists": True, "$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [51]:
# Check that the coordinates and rating value are now numbers

# Find one document to check the data types of 'longitude', 'latitude', and 'RatingValue'
document = establishments.find_one({}, {"geocode.longitude": 1, "geocode.latitude": 1, "RatingValue": 1, "_id": 0})

pprint(document)
print("\nData types:")
print(f"Longitude: {type(document['geocode']['longitude'])}")
print(f"Latitude: {type(document['geocode']['latitude'])}")
print(f"RatingValue: {type(document.get('RatingValue', None))}")

{'RatingValue': 4, 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}}

Data types:
Longitude: <class 'float'>
Latitude: <class 'float'>
RatingValue: <class 'int'>
